## Importing necessary libraries

In [1]:
import nltk
from nltk.util import ngrams
from nltk.corpus import reuters
from nltk.tokenize import word_tokenize
from collections import defaultdict

In [43]:
words = reuters.words()
words = [word.lower() for word in words]
# also remove any punctuation marks present
words = [word for word in words if word.isalpha()]

In [44]:
words[0:100]

['asian',
 'exporters',
 'fear',
 'damage',
 'from',
 'u',
 's',
 'japan',
 'rift',
 'mounting',
 'trade',
 'friction',
 'between',
 'the',
 'u',
 's',
 'and',
 'japan',
 'has',
 'raised',
 'fears',
 'among',
 'many',
 'of',
 'asia',
 's',
 'exporting',
 'nations',
 'that',
 'the',
 'row',
 'could',
 'inflict',
 'far',
 'reaching',
 'economic',
 'damage',
 'businessmen',
 'and',
 'officials',
 'said',
 'they',
 'told',
 'reuter',
 'correspondents',
 'in',
 'asian',
 'capitals',
 'a',
 'u',
 's',
 'move',
 'against',
 'japan',
 'might',
 'boost',
 'protectionist',
 'sentiment',
 'in',
 'the',
 'u',
 's',
 'and',
 'lead',
 'to',
 'curbs',
 'on',
 'american',
 'imports',
 'of',
 'their',
 'products',
 'but',
 'some',
 'exporters',
 'said',
 'that',
 'while',
 'the',
 'conflict',
 'would',
 'hurt',
 'them',
 'in',
 'the',
 'long',
 'run',
 'in',
 'the',
 'short',
 'term',
 'tokyo',
 's',
 'loss',
 'might',
 'be',
 'their',
 'gain',
 'the',
 'u']

In [45]:
words = word_tokenize(' '.join(words))

In [47]:
# check the dimension of words
print(words[0])
len(words)

asian


1327270

## Model definition (without fall-back)

In [48]:
class NGramModel:
    def __init__(self, n, words):
        self.n = n
        self.model = defaultdict(lambda: defaultdict(int))
        self._build_model(words)

    def _build_model(self, words):
        for gram in ngrams(words, self.n):
            context = gram[:-1]
            word = gram[-1]
            self.model[context][word] += 1

        # Convert counts to probabilities
        for context in self.model:
            total = float(sum(self.model[context].values()))
            for word in self.model[context]:
                self.model[context][word] /= total

    def get_next_word_probs(self, *context):
        if len(context) != self.n - 1:
            raise ValueError(f"Expected context length {self.n - 1}, got {len(context)}")

        probs = self.model[context]
        if probs:
            return sorted(probs.items(), key=lambda x: x[1], reverse=True)
        else:
            return []

## Example to show what happens in the backend during the calculation of the next word

In [49]:
# Example: Build a trigram model (n=3)
n = 3
model = NGramModel(n, words)

# Query with context ('the', 'stock')
context = ('the', 'stock')
predictions = model.get_next_word_probs(*context)

# Output
if predictions:
    print(f"Next word probabilities for context {context}:")
    for word, prob in predictions:
        print(f"{word}: {prob:.4f}")
else:
    print("No predictions available for this context.")

Next word probabilities for context ('the', 'stock'):
of: 0.1009
market: 0.0877
for: 0.0746
split: 0.0614
to: 0.0395
was: 0.0395
exchange: 0.0351
is: 0.0307
price: 0.0263
in: 0.0263
dividend: 0.0219
on: 0.0175
has: 0.0175
as: 0.0175
and: 0.0175
at: 0.0175
will: 0.0132
purchases: 0.0132
closed: 0.0132
jumped: 0.0132
the: 0.0088
based: 0.0088
purchase: 0.0088
from: 0.0088
may: 0.0088
today: 0.0088
rose: 0.0088
manager: 0.0044
going: 0.0044
coniston: 0.0044
following: 0.0044
dropped: 0.0044
spectra: 0.0044
including: 0.0044
total: 0.0044
nearly: 0.0044
between: 0.0044
collapse: 0.0044
fall: 0.0044
atlantis: 0.0044
represents: 0.0044
centronics: 0.0044
last: 0.0044
sales: 0.0044
harlow: 0.0044
after: 0.0044
russell: 0.0044
could: 0.0044
under: 0.0044
moved: 0.0044
north: 0.0044
should: 0.0044
depreciation: 0.0044
since: 0.0044
wednesday: 0.0044
metz: 0.0044
fell: 0.0044
goes: 0.0044
subscription: 0.0044
increased: 0.0044
he: 0.0044
consolidation: 0.0044
into: 0.0044
languished: 0.0044
bein

## Checking if can produce a coherent sentence of given length or not

In [50]:
n = 8
context = ('the', 'stocks', 'of')
model = NGramModel(4, words)
sentence = list(context)
for _ in range(n):
    predictions = model.get_next_word_probs(*sentence[-3:])
    if predictions:
        next_word = predictions[0][0]
        sentence.append(next_word)
    else:
        break

print(' '.join(sentence))

the stocks of other textile makers rose along with burlington j


In [51]:
n = 8
context = ('I ', 'feel', 'like')
model = NGramModel(4, words)
sentence = list(context)
for _ in range(n):
    predictions = model.get_next_word_probs(*sentence[-3:])
    if predictions:
        next_word = predictions[0][0]
        sentence.append(next_word)
    else:
        break

print(' '.join(sentence))

I  feel like


## We see that 'I feel like' is unable to move ahead, as it has never seen a text combination like that... we want to avoid that in the real case scenarios. We will now use interpolated N-gram model

In [59]:
class InterpolatedNGramModel:
    def __init__(self, n, words, lambdas=None):
        self.n = n
        self.lambdas = lambdas if lambdas else [1 / n] * n  # Equal weight default
        self.models = [defaultdict(Counter) for _ in range(n)]
        self.vocab = set(words)
        self._build_models(words)

    def _build_models(self, words):
        for k in range(1, self.n + 1):
            for gram in ngrams(words, k):
                context = gram[:-1]
                word = gram[-1]
                self.models[k - 1][context][word] += 1

        # Normalize to probabilities
        for k in range(self.n):
            for context in self.models[k]:
                total = float(sum(self.models[k][context].values()))
                for word in self.models[k][context]:
                    self.models[k][context][word] /= total

    def get_next_word_probs(self, *context):
        """
        Returns interpolated probabilities for next words given context.
        """
        if len(context) > self.n - 1:
            context = context[-(self.n - 1):]  # Use most recent (n-1) words

        scores = defaultdict(float)
        for k in range(self.n):
            subcontext = context[-(k):] if k > 0 else ()
            model_k = self.models[k]
            if subcontext in model_k:
                for word, prob in model_k[subcontext].items():
                    scores[word] += self.lambdas[k] * prob

        # Normalize final scores (optional, but for consistency)
        total_score = sum(scores.values())
        if total_score > 0:
            for word in scores:
                scores[word] /= total_score

        return sorted(scores.items(), key=lambda x: x[1], reverse=True)

# Example usage:
model = InterpolatedNGramModel(n=4, words=words)
context = ('i', 'feel', 'like')
predictions = model.get_next_word_probs(*context)

# Show top 10 predictions
for word, prob in predictions[:10]:
    print(f"{word}: {prob:.4f}")

if: 0.5016
to: 0.0624
the: 0.0471
a: 0.0211
this: 0.0082
in: 0.0080
that: 0.0077
of: 0.0069
it: 0.0059
other: 0.0054


In [61]:
n = 4
context = ('i', 'feel', 'like')
sentence = list(context)
for _ in range(n):
    predictions = model.get_next_word_probs(*sentence[-3:])
    if predictions:
        next_word = predictions[0][0]
        sentence.append(next_word)
    else:
        break

print(' '.join(sentence))

i feel like if we didn t
